In [212]:
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras.models import Sequential
from keras.utils.vis_utils import plot_model
import numpy as np

In [213]:
fin = open("3Ksentcs.txt", 'rb')
lines = []
for line in fin:
    line = line.strip().lower()
    line = line.decode("ascii","ignore")
    if len(line)==0:
        continue
    lines.append(line)
fin.close()
text = " ".join(lines)
specs = "',]\"{?-+_;$*[&)~|=/#}%<:>`!.()1234567890"
for sc in specs:
    text = text.replace(sc, "")

words = text.split(' ')



In [214]:
len(words)

72151

In [215]:
keywords = set([w for w in words])
nb_words = len(keywords)
words2index = dict((w,i) for i, w in enumerate(keywords))
index2words = dict((i,w) for i, w in enumerate(keywords))

In [216]:
len(words2index)

13429

In [217]:
len(index2words)

13429

In [218]:
print(nb_words)

13429


In [219]:
l = list(keywords)
l.sort()

In [220]:
for word in l: 
    if l.index(word)!= len(l)-1:
        nextW = l[l.index(word)+1]        
        if nextW == word+"s" or nextW == word+ "ed":
            for w in text:
                if w == nextW:
                    w = word
            l.remove(nextW)

In [221]:
print(l)

['', 'a', 'ab', 'abalone', 'abandoned', 'abandons', 'abarat', 'abbey', 'abbot', 'abbott', 'abbzug', 'abc', 'abducted', 'abel', 'abercrombie', 'abernethie', 'abhorsens', 'abigail', 'abilities', 'abilitiesare', 'ability', 'ablard', 'able', 'ablett', 'abner', 'abnormally', 'aboard', 'abolished', 'aboriginal', 'abounds', 'about', 'above', 'aboveaverage', 'abrahamic', 'abroad', 'abruc', 'abrupt', 'abruptly', 'absconded', 'absence', 'absent', 'abshire', 'absolute', 'absolutely', 'abundant', 'abuse', 'abused', 'abuses', 'abusive', 'abyssal', 'academic', 'academy', 'accelerator', 'accept', 'acceptance', 'accepted', 'accepts', 'accession', 'accident', 'accidental', 'accidentally', 'accidents', 'accompanied', 'accomplice', 'accomplish', 'accordance', 'according', 'account', 'accumulated', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'achille', 'aching', 'acquaintance', 'acquired', 'acquittal', 'acre', 'across', 'act', 'acting', 'action', 'actionwhich', 'active', 'actively', 'acti

In [222]:
SEQLEN = 10
STEP = 1
input_words = []
label_words = []
for i in range(0, len(words) - SEQLEN, STEP):
    input_words.append(words[i:i+SEQLEN])
    label_words.append(words[i+SEQLEN])
    

In [223]:
X = np.zeros((len(input_words), SEQLEN, nb_words), dtype = np.bool)
y = np.zeros((len(input_words), nb_words), dtype = np.bool)
for i, input_word in enumerate(input_words):
    for j, w in enumerate(input_word):
        X[i,j,words2index[w]] = 1
    y[i,words2index[label_words[i]]]=1

In [224]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCH_PER_ITERATION = 1
NUM_PERDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences = False,
                    input_shape=(SEQLEN,nb_words), unroll = True))
model.add(Dense(nb_words))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [225]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X,y,batch_size=BATCH_SIZE, epochs = NUM_EPOCH_PER_ITERATION)

    test_idx = np.random.randint(len(input_words))
    #print("test_idx: ", test_idx)
    test_words = input_words[test_idx]
    #print("test_words: ", test_words)
    
    print("Generation from seed: %s" % (test_words))
    print(test_words, end = " ")
    for i in range (NUM_PERDS_PER_EPOCH):
        Xtest = np.zeros((1,SEQLEN, nb_words))
        for i, w in enumerate(test_words):
            Xtest[0,i, words2index[w]]=1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2words[np.argmax(pred)]
        print(ypred, end=" ")
        #test_words = test_words[1:].append(ypred)
        test_words.append(ypred)
        #test_words.remove(0)
        del test_words[0]
    print()

Iteration #: 0
Epoch 1/1
41472/72141 [================>.............] - ETA: 18s - loss: 7.2703

KeyboardInterrupt: 

In [ ]:
input_words[test_idx]

In [ ]:
del test_words[0]

In [ ]:
test_words[0]